In [ ]:
import pandas as pd
from itertools import product


In [ ]:
# Load the dataset
file_path = 'asset_1.csv'
data = pd.read_csv(file_path)


In [ ]:
# Task 1: Static Thresholds
def apply_static_thresholds(data, build_threshold=0.6, liquidate_threshold=0.2):
    data['position'] = 0
    current_position = 0

    for i in range(1, len(data)):
        if current_position == 0:
            if data.loc[i, 'alpha'] >= build_threshold:
                current_position = 1
            elif data.loc[i, 'alpha'] <= -build_threshold:
                current_position = -1
        elif current_position == 1:
            if data.loc[i, 'alpha'] <= liquidate_threshold:
                current_position = 0
        elif current_position == -1:
            if data.loc[i, 'alpha'] >= -liquidate_threshold:
                current_position = 0
        data.loc[i, 'position'] = current_position

    return data

# Apply static thresholds
data = apply_static_thresholds(data)


In [ ]:
# Task 2: Backtesting Engine
def backtest_strategy(data):
    data['price_change'] = data['price'].diff()
    data['trade'] = data['position'].diff()
    data['pnl'] = -data['trade'] * data['price_change']
    data['pnl'].fillna(0, inplace=True)
    data['cumulative_pnl'] = data['pnl'].cumsum()
    return data

# Backtest the strategy
data = backtest_strategy(data)


In [ ]:
# Task 3: Optimizing Thresholds
def evaluate_strategy(build_threshold, liquidate_threshold, data):
    data = apply_static_thresholds(data.copy(), build_threshold, liquidate_threshold)
    data = backtest_strategy(data)
    return data['cumulative_pnl'].iloc[-1]

def optimize_thresholds(data, build_thresholds, liquidate_thresholds):
    best_build_threshold = None
    best_liquidate_threshold = None
    best_pnl = float('-inf')

    for build_thresh, liquidate_thresh in product(build_thresholds, liquidate_thresholds):
        pnl = evaluate_strategy(build_thresh, liquidate_thresh, data)
        if pnl > best_pnl:
            best_pnl = pnl
            best_build_threshold = build_thresh
            best_liquidate_threshold = liquidate_thresh

    return best_build_threshold, best_liquidate_threshold, best_pnl


In [ ]:
# Define the range for thresholds
build_thresholds = [x * 0.1 for x in range(5, 11)]  # 0.5 to 1.0
liquidate_thresholds = [x * 0.1 for x in range(1, 5)]  # 0.1 to 0.4


In [4]:
# Optimize thresholds
best_build_threshold, best_liquidate_threshold, best_pnl = optimize_thresholds(data, build_thresholds, liquidate_thresholds)

print("Best Build Threshold:", best_build_threshold)
print("Best Liquidate Threshold:", best_liquidate_threshold)
print("Best P&L:", best_pnl)


Best Build Threshold: 0.7000000000000001
Best Liquidate Threshold: 0.2
Best P&L: 16943.791753721027
